In [1]:
import pandas as pd

path = "fictional_delivery_van_ops_raw_PH_2025.xlsx"
df  = pd.read_excel(path, sheet_name="Raw_Operations_2025")
veh = pd.read_excel(path, sheet_name="Reference", header=17, usecols=[0,1,2], nrows=3)

In [2]:
df = pd.read_excel('fictional_delivery_van_ops_raw_PH_2025.xlsx', sheet_name='Raw_Operations_2025')
df.head(2)

,record_id,operating_date,month_raw,week_num_raw,hub_name,region,sorting_center,service_area_city,route_id,van_id,...,fixed_cost_alloc_php,total_cost_php,cost_per_delivery_php,revenue_php,profit_php,weather,traffic_level,incident_flag,downtime_minutes,notes
0,OPS-202501-00001,07/01/2025,January,2,MM-North Hub,NCR,MNL-SC01,Marikina,NCR-NH-023,VAN-0007,...,₱ 941.44,3322.27,PHP 37.33,"₱ 4,402.21","₱1,079.94",Sunny,Medium,0,0.0,No incident
1,OPS-202501-00002,2025-01-04,2025-01,1,MM-East hub,NCR,MNL-SC02,San Mateo,NCR-EH-027,VAN-0016,...,₱ 572.95,4612.81,₱30.35,"₱9,472.29","₱4,859.48",Heavy Rain,High,0,0.0,No incident


In [3]:
df.shape

(600, 50)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   record_id                 600 non-null    object 
 1   operating_date            600 non-null    object 
 2   month_raw                 600 non-null    object 
 3   week_num_raw              600 non-null    object 
 4   hub_name                  600 non-null    object 
 5   region                    600 non-null    object 
 6   sorting_center            600 non-null    object 
 7   service_area_city         600 non-null    object 
 8   route_id                  600 non-null    object 
 9   van_id                    600 non-null    object 
 10  driver_id                 600 non-null    object 
 11  shift                     600 non-null    object 
 12  vehicle_type              600 non-null    object 
 13  van_age_years             572 non-null    object 
 14  odometer_k

In [5]:
df.columns = df.columns.str.strip().str.lower()

text_cols = ["hub_name","region","sorting_center","service_area_city","route_id","van_id","driver_id",
             "shift","vehicle_type","weather","traffic_level","notes"]
for c in text_cols:
  df[c] = df[c].astype("string").str.strip()

df["hub_name"]     = df["hub_name"].str.title()
df["vehicle_type"] = df["vehicle_type"].str.title()

In [9]:
if "operating_date" in df.columns:
    df["operating_date"] = pd.to_datetime(df["operating_date"], errors="coerce", dayfirst=True)

def to_num(x):
    return pd.to_numeric(
        x.astype("string").str.replace(r"[^0-9.\-]", "", regex=True),
        errors="coerce"
    )

money_cols = ["fuel_cost_php","tolls_php","parking_php","maintenance_php","misc_php",
              "driver_base_pay_php","overtime_pay_php","fixed_cost_alloc_php"]

for c in money_cols:
    if c in df.columns:
        df[c] = to_num(df[c])

num_cols = ["parcels_loaded","delivery_attempts","parcels_delivered","returns_to_sender","damaged_parcels",
            "on_time_deliveries","distance_km","drive_time_hr","stop_time_hr","idle_time_hr","fuel_liters"]

for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

In [10]:
if "record_id" in df.columns:
  df = df.drop_duplicates(subset=["record_id"]).copy()
else:
  df = df.drop_duplicates().copy()

rules = []
if "distance_km" in df.columns: rules.append(df["distance_km"] > 0)
if "fuel_liters" in df.columns: rules.append(df["fuel_liters"] > 0)
if "delivery_attempts" in df.columns and "parcels_delivered" in df.columns:
  rules.append(df["delivery_attempts"] >= df["parcels_delivered"])
if "parcels_delivered" in df.columns: rules.append(df["parcels_delivered"] >= 0)

if rules:
  keep = rules[0]
  for r in rules[1:]:
    keep = keep & r
  df = df.loc[keep].copy()

In [11]:
veh = veh.rename(columns={"vehicle_type_standard":"vehicle_type"})
df  = df.merge(veh, on="vehicle_type", how="left")

df["actual_km_per_liter"] = df["distance_km"] / df["fuel_liters"]
df["fuel_eff_vs_standard"] = df["actual_km_per_liter"] / df["typical_km_per_liter"]
df["over_capacity_flag"] = df["parcels_loaded"] > df["typical_capacity_parcels"]

In [12]:
cost_parts = ["fuel_cost_php","tolls_php","parking_php","maintenance_php","misc_php",
              "driver_base_pay_php","overtime_pay_php","fixed_cost_alloc_php"]
existing_cost_parts = [c for c in cost_parts if c in df.columns]
df["total_trip_cost_php"] = df[existing_cost_parts].sum(axis=1)

df["successful_deliveries"] = (
    df["parcels_delivered"] - df["returns_to_sender"] - df["damaged_parcels"]
)
df["successful_deliveries"] = df["successful_deliveries"].clip(lower=0)

df["active_hours"] = df["drive_time_hr"] + df["stop_time_hr"] + df["idle_time_hr"]
df.loc[df["active_hours"] == 0, "active_hours"] = pd.NA

In [13]:
# Row-level KPI components
df.loc[df["successful_deliveries"] == 0, "successful_deliveries"] = pd.NA
df.loc[df["delivery_attempts"] == 0, "delivery_attempts"] = pd.NA
df.loc[df["parcels_delivered"] == 0, "parcels_delivered"] = pd.NA

df["kpi_cost_per_success"] = df["total_trip_cost_php"] / df["successful_deliveries"]
df["kpi_first_attempt_success"] = df["parcels_delivered"] / df["delivery_attempts"]
df["kpi_on_time_rate"] = df["on_time_deliveries"] / df["parcels_delivered"]
df["kpi_deliveries_per_active_hour"] = df["successful_deliveries"] / df["active_hours"]

# Weekly roll-up for dashboards
df["week"] = df["operating_date"].dt.to_period("W").astype(str)

kpi = (df.groupby(["week","hub_name","route_id"], dropna=False)
         .agg(total_cost_php=("total_trip_cost_php","sum"),
              successful_deliveries=("successful_deliveries","sum"),
              delivery_attempts=("delivery_attempts","sum"),
              parcels_delivered=("parcels_delivered","sum"),
              on_time_deliveries=("on_time_deliveries","sum"),
              active_hours=("active_hours","sum"),
              fuel_eff_vs_standard=("fuel_eff_vs_standard","mean"))
         .reset_index())

# Final 4 KPIs (Power BI-ready)
kpi.loc[kpi["successful_deliveries"] == 0, "successful_deliveries"] = pd.NA
kpi.loc[kpi["delivery_attempts"] == 0, "delivery_attempts"] = pd.NA
kpi.loc[kpi["parcels_delivered"] == 0, "parcels_delivered"] = pd.NA

kpi["KPI1_cost_per_successful_delivery"] = kpi["total_cost_php"] / kpi["successful_deliveries"]
kpi["KPI2_first_attempt_success_rate"]   = kpi["parcels_delivered"] / kpi["delivery_attempts"]
kpi["KPI3_on_time_delivery_rate"]        = kpi["on_time_deliveries"] / kpi["parcels_delivered"]
kpi["KPI4_fuel_efficiency_vs_standard"]  = kpi["fuel_eff_vs_standard"]

kpi.to_csv("kpi_table_powerbi.csv", index=False)

In [14]:
kpi.to_csv("kpi_table_powerbi.csv", index=False)

In [15]:
from google.colab import files
files.download("kpi_table_powerbi.csv")  # use the exact name you see

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>